In [1]:
import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

A:\conda3\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
A:\conda3\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
A:\conda3\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
A:\conda3\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future vers

# Загрузка данных для xgboost

In [5]:
train_df = pd.read_csv("cleaned_train.csv", index_col=0)
train_df = train_df[train_df["full_reads_percent"] < 70]
X = train_df.drop(["publish_date","views","session","depth","full_reads_percent","lem_title","article"], axis=1)
col_to_minmax = [ "PER", "ORG", "MISC", "LOC", "PROPN", "ADV",
                "AUX", "ADJ", "PART", "CCONJ", "SYM", "PRON", "SCONJ", "NUM", "DET",
                "VERB", "NOUN", "ADP", "positive", "neutral", "negative", "skip","speech",  "minute"]
article_texts = []
from tqdm import tqdm
for index, row in tqdm(train_df.iterrows()):
    X.loc[index, 'article_len'] = int(len(row.article))
X = X.drop(col_to_minmax, axis=1)
d_y = train_df["depth"]
frp_y = train_df["full_reads_percent"]
v_y = train_df["views"]
cat_col = ["category","tags","authors"]

6997it [00:02, 2766.52it/s]


# XGBOOST predict

In [6]:
#views
X_train, X_valid, y_train_v, y_valid_v = train_test_split(X, v_y, 
                                                      test_size=0.2, 
                                                      random_state=17)


xgb_v = XGBRegressor(random_state=42, n_jobs=-1, n_estimators=100, learning_rate = 0.05)
xgb_v.fit(X_train, y_train_v)

A:\conda3\lib\site-packages\xgboost\data.py:200: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [7]:
#DEPTH
X_train, X_valid, y_train_d, y_valid_d = train_test_split(X, d_y, 
                                                      test_size=0.2, 
                                                      random_state=17)


xgb_d = XGBRegressor( random_state=42, n_jobs=-1, n_estimators=100, learning_rate = 0.05)#,learning_rate=0.05,depth=10)
xgb_d.fit(X_train, y_train_d)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [8]:
#full_reads_percent
#DEPTH
X_train, X_valid, y_train_frp, y_valid_frp = train_test_split(X, frp_y, 
                                                      test_size=0.2, 
                                                      random_state=17)


xgb_frp = XGBRegressor(random_state=42, n_jobs=-1,  n_estimators=100, learning_rate = 0.05)#,learning_rate=0.05,depth=10)
xgb_frp.fit(X_train, y_train_frp)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
score_views = r2_score(y_valid_v, abs(xgb_v.predict(X_valid)))

A:\conda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [10]:
score_depth = r2_score(y_valid_d, abs(xgb_d.predict(X_valid)))

A:\conda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [11]:
score_frp = r2_score(y_valid_frp, abs(xgb_frp.predict(X_valid)))

A:\conda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [12]:
score_views

0.3952462469900251

In [13]:
score_depth

0.83667829997916

In [14]:
score_frp

0.5179732124109315

In [15]:
score = 0.4 * abs(score_views) + 0.3 * abs(score_depth) + 0.3 * abs(score_frp)

score

0.5644939525130375

# Загрузка данных для lightgbm

In [16]:
train_df = pd.read_csv("cleaned_train.csv", index_col=0)
train_df = train_df[train_df["full_reads_percent"] < 70]
X = train_df.drop(["publish_date","views","session","depth","full_reads_percent","lem_title","article"], axis=1)
col_to_minmax = [ "PER", "ORG", "MISC", "LOC", "PROPN", "ADV",
                "AUX", "ADJ", "PART", "CCONJ", "SYM", "PRON", "SCONJ", "NUM", "DET",
                "VERB", "NOUN", "ADP", "positive", "neutral", "negative", "skip","speech",  "minute"]
article_texts = []
from tqdm import tqdm
for index, row in tqdm(train_df.iterrows()):
    X.loc[index, 'article_len'] = int(len(row.article))
X = X.drop(col_to_minmax, axis=1)
d_y = train_df["depth"]
frp_y = train_df["full_reads_percent"]
v_y = train_df["views"]
cat_col = ["category","tags","authors"]

6997it [00:02, 2707.03it/s]


# LGBM predict

In [17]:
#views
X_train, X_valid, y_train_v, y_valid_v = train_test_split(X, v_y, 
                                                      test_size=0.2, 
                                                      random_state=17)


lgb_v = LGBMRegressor( random_state=42, n_jobs=-1, n_estimators=500, learning_rate = 0.05)
lgb_v.fit(X_train, y_train_v, eval_set=(X_valid, y_valid_v), categorical_feature=cat_col, eval_metric='R2')

A:\conda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
A:\conda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['authors', 'category', 'tags']
  _log_warning('categorical_feature in Dataset is overridden.\n'
A:\conda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
A:\conda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1]	valid_0's l2: 1.17007e+10
[2]	valid_0's l2: 1.13678e+10
[3]	valid_0's l2: 1.10454e+10
[4]	valid_0's l2: 1.07452e+10
[5]	valid_0's l2: 1.04508e+10
[6]	valid_0's l2: 1.01798e+10
[7]	valid_0's l2: 9.90126e+09
[8]	valid_0's l2: 9.66204e+09
[9]	valid_0's l2: 9.43947e+09
[10]	valid_0's l2: 9.20404e+09
[11]	valid_0's l2: 8.99635e+09
[12]	valid_0's l2: 8.78609e+09
[13]	valid_0's l2: 8.59804e+09
[14]	valid_0's l2: 8.39441e+09
[15]	valid_0's l2: 8.22086e+09
[16]	valid_0's l2: 8.05384e+09
[17]	valid_0's l2: 7.8936e+09
[18]	valid_0's l2: 7.75205e+09
[19]	valid_0's l2: 7.60046e+09
[20]	valid_0's l2: 7.46084e+09
[21]	valid_0's l2: 7.3257e+09
[22]	valid_0's l2: 7.18257e+09
[23]	valid_0's l2: 7.05414e+09
[24]	valid_0's l2: 6.92378e+09
[25]	valid_0's l2: 6.80192e+09
[26]	valid_0's l2: 6.69621e+09
[27]	valid_0's l2: 6.58401e+09
[28]	valid_0's l2: 6.47301e+09
[29]	valid_0's l2: 6.36791e+09
[30]	valid_0's l2: 6.27319e+09
[31]	valid_0's l2: 6.17738e+09
[32]	valid_0's l2: 6.07656e+09
[33]	valid_0's l2: 

LGBMRegressor(learning_rate=0.05, n_estimators=500, random_state=42)

In [18]:
#DEPTH
X_train, X_valid, y_train_d, y_valid_d = train_test_split(X, d_y, 
                                                      test_size=0.2, 
                                                      random_state=17)


lgb_d = LGBMRegressor( random_state=42, n_jobs=-1, n_estimators=500, learning_rate = 0.05)#,learning_rate=0.05,depth=10)
lgb_d.fit(X_train, y_train_d, eval_set=(X_valid, y_valid_d), categorical_feature=cat_col, eval_metric='R2')

[1]	valid_0's l2: 0.00386844
[2]	valid_0's l2: 0.00361685
[3]	valid_0's l2: 0.00338575
[4]	valid_0's l2: 0.00317546
[5]	valid_0's l2: 0.00298551
[6]	valid_0's l2: 0.00281455
[7]	valid_0's l2: 0.00266074
[8]	valid_0's l2: 0.00251606
[9]	valid_0's l2: 0.00238569
[10]	valid_0's l2: 0.00226793
[11]	valid_0's l2: 0.0021623
[12]	valid_0's l2: 0.0020648
[13]	valid_0's l2: 0.00197344
[14]	valid_0's l2: 0.00189059
[15]	valid_0's l2: 0.00181439
[16]	valid_0's l2: 0.00174406
[17]	valid_0's l2: 0.001682
[18]	valid_0's l2: 0.00162274
[19]	valid_0's l2: 0.00156994
[20]	valid_0's l2: 0.00152387
[21]	valid_0's l2: 0.00147742
[22]	valid_0's l2: 0.00143339
[23]	valid_0's l2: 0.00139643
[24]	valid_0's l2: 0.00136217
[25]	valid_0's l2: 0.00133036
[26]	valid_0's l2: 0.00130006
[27]	valid_0's l2: 0.00127355
[28]	valid_0's l2: 0.00124724
[29]	valid_0's l2: 0.00122231
[30]	valid_0's l2: 0.00120235
[31]	valid_0's l2: 0.00118165
[32]	valid_0's l2: 0.00116251
[33]	valid_0's l2: 0.00114455
[34]	valid_0's l2: 0.00

LGBMRegressor(learning_rate=0.05, n_estimators=500, random_state=42)

In [19]:
#full_reads_percent
#DEPTH
X_train, X_valid, y_train_frp, y_valid_frp = train_test_split(X, frp_y, 
                                                      test_size=0.2, 
                                                      random_state=17)


lgb_frp = LGBMRegressor(boosting_type="gbdt", random_state=42, n_jobs=-1,  n_estimators=500, learning_rate = 0.05)#,learning_rate=0.05,depth=10)
lgb_frp.fit(X_train, y_train_frp, eval_set=(X_valid, y_valid_frp), categorical_feature=cat_col, eval_metric='R2')

[1]	valid_0's l2: 94.8469
[2]	valid_0's l2: 90.7878
[3]	valid_0's l2: 86.9366
[4]	valid_0's l2: 83.5551
[5]	valid_0's l2: 80.5184
[6]	valid_0's l2: 77.8245
[7]	valid_0's l2: 75.361
[8]	valid_0's l2: 73.2135
[9]	valid_0's l2: 71.1672
[10]	valid_0's l2: 69.1585
[11]	valid_0's l2: 67.5118
[12]	valid_0's l2: 65.9142
[13]	valid_0's l2: 64.5151
[14]	valid_0's l2: 63.3137
[15]	valid_0's l2: 62.1422
[16]	valid_0's l2: 61.0592
[17]	valid_0's l2: 60.1391
[18]	valid_0's l2: 59.2588
[19]	valid_0's l2: 58.4593
[20]	valid_0's l2: 57.7857
[21]	valid_0's l2: 57.0084
[22]	valid_0's l2: 56.3623
[23]	valid_0's l2: 55.7732
[24]	valid_0's l2: 55.2982
[25]	valid_0's l2: 54.7695
[26]	valid_0's l2: 54.3071
[27]	valid_0's l2: 53.8383
[28]	valid_0's l2: 53.4936
[29]	valid_0's l2: 53.1173
[30]	valid_0's l2: 52.7805
[31]	valid_0's l2: 52.4662
[32]	valid_0's l2: 52.1335
[33]	valid_0's l2: 51.8994
[34]	valid_0's l2: 51.6008
[35]	valid_0's l2: 51.3204
[36]	valid_0's l2: 51.0905
[37]	valid_0's l2: 50.9344
[38]	valid_

KeyboardInterrupt: 

In [ ]:
score_views = r2_score(y_valid_v, abs(lgb_v.predict(X_valid)))
score_depth = r2_score(y_valid_d, abs(lgb_d.predict(X_valid)))
score_frp = r2_score(y_valid_frp, abs(lgb_frp.predict(X_valid)))

In [ ]:
score_views

In [ ]:
score_depth

In [ ]:
score_frp

In [ ]:
score = 0.4 * abs(score_views) + 0.3 * abs(score_depth) + 0.3 * abs(score_frp)

score

# Загрузка данных для настройки CatBoostRegressor с помощью optuna

In [6]:
import optuna

In [7]:
train_df = pd.read_csv("cleaned_train.csv", index_col=0)
train_df = train_df[train_df["full_reads_percent"] < 70]
X = train_df.drop(["publish_date","views","session","depth","full_reads_percent","lem_title","article"], axis=1)
col_to_minmax = ["PER", "ORG","LOC", "MISC","PROPN", "ADV",
                "AUX", "ADJ", "PART", "CCONJ", "SYM", "PRON", "SCONJ", "NUM", "DET",
                "VERB", "NOUN", "ADP", "minute","year"]

X = X.drop(col_to_minmax, axis=1)
d_y = train_df["depth"]
frp_y = train_df["full_reads_percent"]
v_y = train_df["views"]
cat_col = ["category","tags","authors","day", "mounth","hour"]

In [10]:
SAMPLE_RATE = 0.4
RANDOM_SEED = 1
EARLY_STOPPING_ROUND = 100

## depth tuning

In [11]:

X_train, X_valid, y_train_d, y_valid_d = train_test_split(X, d_y, 
                                                      test_size=0.2, 
                                                      random_state=17)
#cat_col = ["category", "authors", "tags"]
pool_train = Pool(X_train, y_train_d, cat_features=cat_col)
pool_test_d = Pool(X_valid, cat_features=cat_col)
pool_eval = Pool(X_valid, y_valid_d, cat_features=cat_col)

In [12]:
def objective_depth(trial):
    param = {}
    param['depth'] = trial.suggest_int('depth', 2, 10)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = trial.suggest_categorical('iterations', [400,500,600,700,800])
    param['use_best_model'] = True
    param['eval_metric'] = 'R2'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = RANDOM_SEED
    param['logging_level'] = 'Silent'
    
    regressor = CatBoostRegressor(**param)
    
    
    
    regressor.fit(pool_train, eval_set=pool_eval, logging_level='Silent',early_stopping_rounds=EARLY_STOPPING_ROUND)
    r2_views = r2_score(y_valid_d, abs(regressor.predict(pool_test_d)))
    return r2_views

In [13]:
%%time
study = optuna.create_study(study_name=f'catboost-seed{RANDOM_SEED}')
optuna.logging.set_verbosity(optuna.logging.INFO)
study.optimize(objective_depth, n_trials=30, timeout=24000)

[I 2022-07-19 17:47:46,349] A new study created in memory with name: catboost-seed1
[I 2022-07-19 17:54:06,648] Trial 0 finished with value: 0.8325781480926172 and parameters: {'depth': 7, 'l2_leaf_reg': 2.0, 'min_child_samples': 4, 'iterations': 600}. Best is trial 0 with value: 0.8325781480926172.
[I 2022-07-19 17:54:36,170] Trial 1 finished with value: 0.8054416286243039 and parameters: {'depth': 3, 'l2_leaf_reg': 5.5, 'min_child_samples': 16, 'iterations': 500}. Best is trial 1 with value: 0.8054416286243039.
[I 2022-07-19 18:04:50,585] Trial 2 finished with value: 0.8299186956965496 and parameters: {'depth': 9, 'l2_leaf_reg': 3.0, 'min_child_samples': 4, 'iterations': 500}. Best is trial 1 with value: 0.8054416286243039.
[I 2022-07-19 18:05:08,076] Trial 3 finished with value: 0.7703323957465065 and parameters: {'depth': 2, 'l2_leaf_reg': 5.5, 'min_child_samples': 32, 'iterations': 600}. Best is trial 3 with value: 0.7703323957465065.
[I 2022-07-19 18:05:48,086] Trial 4 finished w

KeyboardInterrupt: 

## views tuning

In [14]:

X_train, X_valid, y_train_v, y_valid_v = train_test_split(X, v_y, 
                                                      test_size=0.2, 
                                                      random_state=17)
#cat_col = ["category", "authors", "tags"]
pool_train = Pool(X_train, y_train_v, cat_features=cat_col)
pool_test_v = Pool(X_valid, cat_features=cat_col)
pool_eval = Pool(X_valid, y_valid_v, cat_features=cat_col)

In [15]:
def objective_views(trial):
    param = {}
    param['depth'] = trial.suggest_int('depth', 2, 10)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = trial.suggest_categorical('iterations', [400,500,600,700,800])
    param['use_best_model'] = True
    param['eval_metric'] = 'R2'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = RANDOM_SEED
    param['logging_level'] = 'Silent'
    
    regressor = CatBoostRegressor(**param)
    
    
    
    regressor.fit(pool_train, eval_set=pool_eval, logging_level='Silent',early_stopping_rounds=EARLY_STOPPING_ROUND)
    r2_views = r2_score(y_valid_v, abs(regressor.predict(pool_test_v)))
    return r2_views

In [ ]:
%%time
study = optuna.create_study(study_name=f'catboost-seed{RANDOM_SEED}')
optuna.logging.set_verbosity(optuna.logging.INFO)
study.optimize(objective_views, n_trials=30, timeout=24000)

[I 2022-07-19 18:37:44,459] A new study created in memory with name: catboost-seed1
[I 2022-07-19 18:38:05,464] Trial 0 finished with value: 0.8570005585324214 and parameters: {'depth': 2, 'l2_leaf_reg': 5.5, 'min_child_samples': 8, 'iterations': 500}. Best is trial 0 with value: 0.8570005585324214.


## frp tuning

In [ ]:

X_train, X_valid, y_train_f, y_valid_f = train_test_split(X, frp_y, 
                                                      test_size=0.2, 
                                                      random_state=17)
#cat_col = ["category", "authors", "tags"]
pool_train = Pool(X_train, y_train_f, cat_features=cat_col)
pool_test_f = Pool(X_valid, cat_features=cat_col)
pool_eval = Pool(X_valid, y_valid_f, cat_features=cat_col)

In [ ]:
def objective_frp(trial):
    param = {}
    param['depth'] = trial.suggest_int('depth', 2, 10)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = trial.suggest_categorical('iterations', [400,500,600,700,800])
    param['use_best_model'] = True
    param['eval_metric'] = 'R2'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = RANDOM_SEED
    param['logging_level'] = 'Silent'
    
    regressor = CatBoostRegressor(**param)
    
    
    
    regressor.fit(pool_train, eval_set=pool_eval, logging_level='Silent',early_stopping_rounds=EARLY_STOPPING_ROUND)
    r2_views = r2_score(y_valid_f, abs(regressor.predict(pool_test_f)))
    return r2_views

In [ ]:
%%time
study = optuna.create_study(study_name=f'catboost-seed{RANDOM_SEED}')
optuna.logging.set_verbosity(optuna.logging.INFO)
study.optimize(objective_frp, n_trials=30, timeout=24000)

# Загрузка тренировочных данных для CatBoostRegressor

In [2]:

train_df = pd.read_csv("cleaned_train.csv", index_col=0)
train_df = train_df[train_df["full_reads_percent"] < 70]

X = train_df.drop(["publish_date","views","session","depth","full_reads_percent","lem_title","article"], axis=1)
col_to_minmax = ["minute","genre","keyfeatures","meta_address","aggregator", "year","categ","new_authors","authors"]

X = X.drop(col_to_minmax, axis=1)
d_y = train_df["depth"]
d_y = np.log(d_y)
frp_y = train_df["full_reads_percent"]
v_y = train_df["views"]
cat_col = ["category","tags","day", "mounth","data_type","header_cat","hour"]


In [20]:
X

,ctr,category,tags,day,mounth,hour,month_end,month_start,prime_time,morning,...,юлия_выродова,юлия_кошкина,юлия_криган,юлия_сапронова,article_len,title_len,keyfeatures_len,article_word_count,title_word_count,keyfeatures_word_count
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,1.580,2,5133,4,4,10,0.0,1.0,0.0,1.0,...,0,0,0,0,1440.0,67.0,241.0,163.0,8.0,20.0
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,1.853,0,4325,18,2,10,0.0,0.0,0.0,1.0,...,0,0,0,0,1404.0,70.0,239.0,166.0,8.0,20.0
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,0.000,0,2877,12,2,4,0.0,0.0,0.0,1.0,...,0,0,0,0,1120.0,65.0,244.0,129.0,9.0,20.0
6262a5889a79470b78c9ca307UKY2SSZTjCcjhwBzxw37w,0.000,0,1911,22,4,13,0.0,0.0,0.0,0.0,...,0,0,0,0,1540.0,72.0,226.0,183.0,10.0,20.0
626678929a79477ca0101568wuMYES90REuV5YhrN75IXg,0.000,5,2399,25,4,10,0.0,0.0,0.0,1.0,...,0,0,0,0,2421.0,67.0,230.0,270.0,8.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61fd479c9a7947848183694751RC_1i5QlGuv7B2FzwTjw,2.354,5,6466,4,2,16,0.0,1.0,0.0,0.0,...,0,0,0,0,711.0,66.0,244.0,86.0,9.0,20.0
628a8f1c9a79478c64a7dae450tIkSGVSf2O8XQgk_uaHQ,4.112,3,3248,22,5,20,0.0,0.0,1.0,0.0,...,0,0,0,0,1601.0,69.0,245.0,197.0,9.0,20.0
6244cf5e9a79479099346b26tKlqoxQYSmij0ZBrGzebug,5.125,5,6252,30,3,21,1.0,0.0,1.0,0.0,...,0,0,0,0,700.0,53.0,233.0,85.0,8.0,20.0
627303699a794749b8116abc12JX_3ZDTDu2XHF3k2eJVg,0.000,1,6197,4,5,23,0.0,1.0,1.0,0.0,...,0,0,0,0,1404.0,74.0,233.0,177.0,11.0,20.0


# CatBoostRegression 

## Depth

In [3]:
X_train, X_valid, y_train_d, y_valid_d = train_test_split(X, d_y, 
                                                      test_size=0.2, 
                                                      random_state=17)
pool_train = Pool(X_train, y_train_d, cat_features=cat_col)
pool_test_d = Pool(X_valid, cat_features=cat_col)
pool_eval = Pool(X_valid, y_valid_d, cat_features=cat_col)
cbt_d = CatBoostRegressor(eval_metric='R2', random_state=42, iterations= 600,loss_function='RMSE')
cbt_d.fit(pool_train, eval_set=pool_eval, logging_level='Verbose', early_stopping_rounds=100)

Learning rate set to 0.087713
0:	learn: 0.1105357	test: 0.1153380	best: 0.1153380 (0)	total: 354ms	remaining: 3m 32s
1:	learn: 0.1983602	test: 0.2086998	best: 0.2086998 (1)	total: 507ms	remaining: 2m 31s
2:	learn: 0.2792512	test: 0.2881247	best: 0.2881247 (2)	total: 688ms	remaining: 2m 17s
3:	learn: 0.3505065	test: 0.3603149	best: 0.3603149 (3)	total: 915ms	remaining: 2m 16s
4:	learn: 0.4124091	test: 0.4204660	best: 0.4204660 (4)	total: 1.21s	remaining: 2m 23s
5:	learn: 0.4610318	test: 0.4711450	best: 0.4711450 (5)	total: 1.4s	remaining: 2m 18s
6:	learn: 0.5068452	test: 0.5174010	best: 0.5174010 (6)	total: 1.58s	remaining: 2m 13s
7:	learn: 0.5442174	test: 0.5547861	best: 0.5547861 (7)	total: 1.95s	remaining: 2m 24s
8:	learn: 0.5743430	test: 0.5826781	best: 0.5826781 (8)	total: 2.19s	remaining: 2m 23s
9:	learn: 0.6023076	test: 0.6133061	best: 0.6133061 (9)	total: 2.34s	remaining: 2m 18s
10:	learn: 0.6271326	test: 0.6398079	best: 0.6398079 (10)	total: 2.52s	remaining: 2m 15s
11:	learn: 0

In [19]:
cbt_d.get_feature_importance(data=Pool(X_train, y_train_d, cat_features=cat_col),prettified=True).to_csv("d_imp.csv")

# Views

In [4]:
X_train, X_valid, y_train_v, y_valid_v = train_test_split(X, v_y, 
                                                      test_size=0.2, 
                                                      random_state=17)
pool_train = Pool(X_train, y_train_v, cat_features=cat_col)
pool_test_v = Pool(X_valid, cat_features=cat_col)
pool_eval = Pool(X_valid, y_valid_v, cat_features=cat_col)

cbt_v = CatBoostRegressor(eval_metric='R2', random_state=42, iterations= 700,loss_function='RMSE')
cbt_v.fit(pool_train, eval_set=pool_eval, logging_level='Verbose', early_stopping_rounds=100)

Learning rate set to 0.063804
0:	learn: 0.0390961	test: 0.0539010	best: 0.0539010 (0)	total: 218ms	remaining: 3m 37s
1:	learn: 0.0951535	test: 0.1134563	best: 0.1134563 (1)	total: 421ms	remaining: 3m 30s
2:	learn: 0.1406161	test: 0.1647795	best: 0.1647795 (2)	total: 647ms	remaining: 3m 34s
3:	learn: 0.1897864	test: 0.2154078	best: 0.2154078 (3)	total: 873ms	remaining: 3m 37s
4:	learn: 0.2342933	test: 0.2636169	best: 0.2636169 (4)	total: 1.09s	remaining: 3m 36s
5:	learn: 0.2740377	test: 0.3059912	best: 0.3059912 (5)	total: 1.29s	remaining: 3m 34s
6:	learn: 0.3066656	test: 0.3091580	best: 0.3091580 (6)	total: 1.51s	remaining: 3m 34s
7:	learn: 0.3449563	test: 0.3501397	best: 0.3501397 (7)	total: 1.74s	remaining: 3m 36s
8:	learn: 0.3742421	test: 0.3827981	best: 0.3827981 (8)	total: 1.93s	remaining: 3m 32s
9:	learn: 0.3995365	test: 0.4143172	best: 0.4143172 (9)	total: 2.14s	remaining: 3m 31s
10:	learn: 0.4211704	test: 0.4285205	best: 0.4285205 (10)	total: 2.35s	remaining: 3m 31s
11:	learn: 

In [21]:
cbt_v.get_feature_importance(data=Pool(X_train, y_train_v, cat_features=cat_col),prettified=True).to_csv("v_imp.csv")

# Full Reads Percent

In [7]:
X_train, X_valid, y_train_frp, y_valid_frp = train_test_split(X, frp_y, 
                                                      test_size=0.2, 
                                                      random_state=17)
pool_train = Pool(X_train, y_train_frp, cat_features=cat_col)
pool_test_frp = Pool(X_valid, cat_features=cat_col)
pool_eval = Pool(X_valid, y_valid_frp, cat_features=cat_col)

cbt_frp = CatBoostRegressor(eval_metric='R2', random_state=42,iterations = 400,loss_function='RMSE')
cbt_frp.fit(pool_train, eval_set=pool_eval, logging_level='Verbose', early_stopping_rounds = 100)

Learning rate set to 0.112919
0:	learn: 0.0893192	test: 0.0843655	best: 0.0843655 (0)	total: 180ms	remaining: 1m 11s
1:	learn: 0.1574231	test: 0.1531418	best: 0.1531418 (1)	total: 340ms	remaining: 1m 7s
2:	learn: 0.2164696	test: 0.2110780	best: 0.2110780 (2)	total: 498ms	remaining: 1m 5s
3:	learn: 0.2636874	test: 0.2583053	best: 0.2583053 (3)	total: 645ms	remaining: 1m 3s
4:	learn: 0.3038126	test: 0.2981583	best: 0.2981583 (4)	total: 803ms	remaining: 1m 3s
5:	learn: 0.3389628	test: 0.3325149	best: 0.3325149 (5)	total: 944ms	remaining: 1m 1s
6:	learn: 0.3654816	test: 0.3577903	best: 0.3577903 (6)	total: 1.11s	remaining: 1m 2s
7:	learn: 0.3880298	test: 0.3789868	best: 0.3789868 (7)	total: 1.27s	remaining: 1m 2s
8:	learn: 0.4075524	test: 0.3984227	best: 0.3984227 (8)	total: 1.51s	remaining: 1m 5s
9:	learn: 0.4227747	test: 0.4132762	best: 0.4132762 (9)	total: 1.65s	remaining: 1m 4s
10:	learn: 0.4383280	test: 0.4270308	best: 0.4270308 (10)	total: 1.84s	remaining: 1m 4s
11:	learn: 0.4486310	

In [23]:
cbt_frp.get_feature_importance(data=Pool(X_train, y_train_frp, cat_features=cat_col),prettified=True).to_csv("fpr_imp.csv")

In [8]:
score_views = r2_score(abs(y_valid_v), abs(cbt_v.predict(pool_test_v)))
score_depth = r2_score(np.exp(y_valid_d), np.exp(cbt_d.predict(pool_test_d)))
score_frp = r2_score(abs(y_valid_frp), abs(cbt_frp.predict(pool_test_frp)))

In [9]:
score_views

0.8851029590572099

In [10]:
score_depth

0.8548810720404549

In [11]:
score_frp

0.5647865907512027

In [12]:
score = 0.4 * abs(score_views) + 0.3 * abs(score_depth) + 0.3 * abs(score_frp)

score

0.7799414824603813

# Предсказание результата (РЕШЕНИЕ ЗАДАЧИ)

In [13]:

test_df = pd.read_csv("cleaned_test.csv", index_col=0)
cat_col = ["category","tags","day", "mounth","data_type","header_cat","hour"]



In [14]:
test = test_df.drop(["publish_date","session","lem_title","article", "minute","genre","keyfeatures","meta_address","aggregator", "year","categ","new_authors","authors"], axis=1)
test_index = test.index
test_index

Index(['61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw',
       '628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw',
       '627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw',
       '628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ',
       '620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw',
       '6267e9d39a79470f677fb6fajv_4Eb5ZQtip4IbvDqDuKw',
       '624925809a79476c4461de30m1RFLN6XSPyEGdKhDO5nyA',
       '6239e4a39a7947e926f82194t8mN2pE6R4mAGowkxvegNw',
       '628a1a4e9a794771555c208bICJ13mlxTMehQcRuqtV6YA',
       '626a5b0a9a7947627960acd0TdQDl3OoSMO5YuE7-m19eQ',
       ...
       '624172f09a7947d8a1be27ffZqSJ76KpT52AHNR4akYo2g',
       '6292c0e59a794731e7638eafl-W6BG-xRmee_FmiCQZy3g',
       '6251b7c09a794726445ad5814uajlrCQRA-qDAXZoDdNTg',
       '628dbbfc9a794710bc23acfcfiQ-2pywRXq9U_G6feLy9A',
       '624b49b89a794712b4d8a169mBvBwIt2RkuR5Er1QfHVUw',
       '623ba6bc9a79470214cb9418x4OZQ1IjTb27nPAgv5wAZg',
       '624fda1f9a79471a9bd102eetuRTrLPGSZyNbc9_YcBAeQ',
       '6257a6a99a79

In [15]:

train_df = pd.read_csv("cleaned_train.csv", index_col=0)
train_df = train_df[train_df["full_reads_percent"] < 70]

X = train_df.drop(["publish_date","views","session","depth","full_reads_percent","lem_title","article"], axis=1)
col_to_minmax = ["minute","genre","keyfeatures","meta_address","aggregator", "year","categ","new_authors","authors"]

X = X.drop(col_to_minmax, axis=1)
d_y = train_df["depth"]
d_y = np.log(d_y)
frp_y = train_df["full_reads_percent"]
v_y = train_df["views"]
cat_col = ["category","tags","day", "mounth","data_type","header_cat","hour"]


## depth

In [16]:

pool_train_d = Pool(X, d_y, cat_features=cat_col)
pool_test = Pool(test, cat_features=cat_col)
cbt_d = CatBoostRegressor(eval_metric='R2', random_state=42, iterations= 400,loss_function='RMSE')
cbt_d.fit(pool_train_d, logging_level='Verbose', early_stopping_rounds=20)
pred_d = np.exp(cbt_d.predict(pool_test))

Learning rate set to 0.110721
0:	learn: 0.1479968	total: 177ms	remaining: 1m 10s
1:	learn: 0.2629310	total: 356ms	remaining: 1m 10s
2:	learn: 0.3595365	total: 530ms	remaining: 1m 10s
3:	learn: 0.4376760	total: 689ms	remaining: 1m 8s
4:	learn: 0.5003398	total: 832ms	remaining: 1m 5s
5:	learn: 0.5506861	total: 1.04s	remaining: 1m 8s
6:	learn: 0.5933653	total: 1.18s	remaining: 1m 6s
7:	learn: 0.6278974	total: 1.35s	remaining: 1m 6s
8:	learn: 0.6573779	total: 1.51s	remaining: 1m 5s
9:	learn: 0.6784088	total: 1.72s	remaining: 1m 6s
10:	learn: 0.6973005	total: 1.86s	remaining: 1m 5s
11:	learn: 0.7152100	total: 2.07s	remaining: 1m 6s
12:	learn: 0.7277221	total: 2.26s	remaining: 1m 7s
13:	learn: 0.7393400	total: 2.42s	remaining: 1m 6s
14:	learn: 0.7481642	total: 2.58s	remaining: 1m 6s
15:	learn: 0.7577047	total: 2.73s	remaining: 1m 5s
16:	learn: 0.7652302	total: 2.92s	remaining: 1m 5s
17:	learn: 0.7711203	total: 3.07s	remaining: 1m 5s
18:	learn: 0.7763405	total: 3.27s	remaining: 1m 5s
19:	lear

## Views

In [17]:
pool_train_v = Pool(X, v_y, cat_features=cat_col)
pool_test = Pool(test, cat_features=cat_col)
cbt_v = CatBoostRegressor(eval_metric='R2', random_state=42, iterations= 800,loss_function='RMSE')
cbt_v.fit(pool_train_v, logging_level='Verbose', early_stopping_rounds=20)
pred_v = abs(cbt_v.predict(pool_test))

Learning rate set to 0.06538
0:	learn: 0.0570988	total: 156ms	remaining: 2m 4s
1:	learn: 0.1101934	total: 329ms	remaining: 2m 11s
2:	learn: 0.1619472	total: 485ms	remaining: 2m 8s
3:	learn: 0.2161550	total: 657ms	remaining: 2m 10s
4:	learn: 0.2656433	total: 811ms	remaining: 2m 9s
5:	learn: 0.3100181	total: 974ms	remaining: 2m 8s
6:	learn: 0.3495399	total: 1.11s	remaining: 2m 6s
7:	learn: 0.3870421	total: 1.27s	remaining: 2m 5s
8:	learn: 0.4212092	total: 1.41s	remaining: 2m 3s
9:	learn: 0.4468612	total: 1.58s	remaining: 2m 4s
10:	learn: 0.4701937	total: 1.72s	remaining: 2m 3s
11:	learn: 0.4988513	total: 1.9s	remaining: 2m 4s
12:	learn: 0.5267117	total: 2.04s	remaining: 2m 3s
13:	learn: 0.5437054	total: 2.19s	remaining: 2m 2s
14:	learn: 0.5543056	total: 2.35s	remaining: 2m 2s
15:	learn: 0.5758146	total: 2.53s	remaining: 2m 3s
16:	learn: 0.5907904	total: 2.68s	remaining: 2m 3s
17:	learn: 0.6079320	total: 2.83s	remaining: 2m 2s
18:	learn: 0.6251786	total: 2.97s	remaining: 2m 2s
19:	learn: 

## frp

In [18]:
pool_train_frp = Pool(X, frp_y, cat_features=cat_col)
pool_test = Pool(test, cat_features=cat_col)
cbt_frp = CatBoostRegressor(eval_metric='R2', random_state=42, iterations= 300,loss_function='RMSE')
cbt_frp.fit(pool_train_frp, logging_level='Verbose', early_stopping_rounds=20)
pred_frp = abs(cbt_frp.predict(pool_test))

Learning rate set to 0.137779
0:	learn: 0.0985476	total: 176ms	remaining: 52.7s
1:	learn: 0.1832524	total: 337ms	remaining: 50.3s
2:	learn: 0.2485041	total: 506ms	remaining: 50.1s
3:	learn: 0.2959884	total: 665ms	remaining: 49.2s
4:	learn: 0.3364972	total: 836ms	remaining: 49.3s
5:	learn: 0.3659022	total: 1s	remaining: 49.1s
6:	learn: 0.3921817	total: 1.17s	remaining: 49.1s
7:	learn: 0.4120446	total: 1.32s	remaining: 48s
8:	learn: 0.4295926	total: 1.48s	remaining: 47.8s
9:	learn: 0.4437266	total: 1.63s	remaining: 47.1s
10:	learn: 0.4568620	total: 1.79s	remaining: 47s
11:	learn: 0.4667075	total: 1.97s	remaining: 47.4s
12:	learn: 0.4747342	total: 2.13s	remaining: 47s
13:	learn: 0.4828406	total: 2.28s	remaining: 46.7s
14:	learn: 0.4877339	total: 2.43s	remaining: 46.1s
15:	learn: 0.4953618	total: 2.59s	remaining: 45.9s
16:	learn: 0.5023126	total: 2.73s	remaining: 45.4s
17:	learn: 0.5074811	total: 2.88s	remaining: 45.2s
18:	learn: 0.5118423	total: 3.03s	remaining: 44.8s
19:	learn: 0.5157757

In [19]:
df_submission_extended = pd.DataFrame(
    {'views': pred_v, 'depth': pred_d, 'full_reads_percent': pred_frp}, 
    index=test_index,
)
df_submission_extended.to_csv('submit21.csv')